In [ ]:
spark

In [ ]:
sqlContext

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
spark = SparkSession(sc)

In [ ]:
import re
import pandas as pd

In [ ]:
import glob

source = glob.glob('*.gz')
source

In [ ]:
dfNasa = sqlContext.read.text(source)
dfNasa.printSchema()

In [ ]:
type(dfNasa)

In [ ]:
dfNasa.show(10, truncate=False)

In [ ]:
smpLogs = [item['value'] for item in dfNasa.take(15)]
smpLogs

In [ ]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
host = [re.search(host_pattern, item).group(1)
           if re.search(host_pattern, item)
           else 'no match'
           for item in smpLogs]
host

In [ ]:
ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
timestamp = [re.search(ts_pattern, item).group(1) for item in smpLogs]
timestamp

In [ ]:
requisicao_ptrn = r'\"(\S+)\s(\S+)\s*(\S*)\"'
requisicao = [re.search(requisicao_ptrn, item).groups()
               if re.search(requisicao_ptrn, item)
               else 'no match'
              for item in smpLogs]
requisicao

In [ ]:
cod_ret_http_ptrn = r'\s(\d{3})\s'
cod_ret_http = [re.search(cod_ret_http_ptrn, item).group(1) for item in smpLogs]
print(cod_ret_http)

In [ ]:
bytes_retorn_ptrn = r'\s(\d+)$'
bytes_retorn = [re.search(bytes_retorn_ptrn, item).group(1) for item in smpLogs]
print(bytes_retorn)

In [ ]:
from pyspark.sql.functions import regexp_extract

In [ ]:
logDf = dfNasa.select(regexp_extract('value', host_pattern, 1).alias('host'),
                      regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                      regexp_extract('value', requisicao_ptrn, 2).alias('requisicao'),
                      regexp_extract('value', cod_ret_http_ptrn, 1).cast('integer').alias('cod_ret_http'),
                      regexp_extract('value', bytes_retorn_ptrn, 1).cast('integer').alias('bytes'))

In [ ]:
logDf.show(10, truncate=True)

In [ ]:
from pyspark.sql.functions import col, countDistinct, count

In [ ]:
# 1. Número de hosts únicos
logDf.groupBy(host).count().show()

In [ ]:
# 2. O total de erros 404. 
df_tot = spark.sql("Select COUNT(*) From logDf Where cod_ret_http = '404'")

In [ ]:
# 3. Os 5 URLs que mais causaram erro 404.
df_top5 = spark.sql("Select host, COUNT(*) From logDf Where cod_ret_http = '404'")

In [ ]:
# 4. Quantidade de erros 404 por dia.
df_err = spark.sql("Select host, COUNT(*) From logDf Where cod_ret_http = '404' Group By timestamp")

In [ ]:
# 5. O total de bytes retornados.
df_bytes = logDf.agg({"bytes":"sum"}).collect()[0]